In [1]:
from matplotlib import pyplot as plt
import sys
import requests
import base64
import json
from pprint import pprint
from dotenv import load_dotenv
import os

sys.path.append('../../..')
from hydradx.model.processing import query_sqlPad

In [2]:
# query = (
#     f"with fees as ("
#     f"select "
#     f"    timestamp,"
#     f"    operation,"
#     f"    asset_fee_rate,"
#     f"    protocol_fee_rate,"
#     f"    amount,"
#     f"    asset_fee"
#     f"from"
#     f"(select "
#     f"    timestamp,"
#     f"    block,"
#     f"    who,"
#     f"    operation,"
#     f"    (select symbol from token_metadata where id = asset_in limit 1) as asset_in,"
#     f"    (select symbol from token_metadata where id = asset_out limit 1) as asset_out,"
#     f"    asset_fee / amount * 100 as asset_fee_rate,"
#     f"    protocol_fee / amount * 100 as protocol_fee_rate,"
#     f"    amount,"
#     f"    amount_in,"
#     f"    asset_fee"
#     f"from "
#     f"(select "
#     f"    timestamp,"
#     f"    block.height as block,"
#     f"    args->>'who' as who,"
#     f"    name as operation,"
#     f"    (args->>'assetIn')::integer as asset_in, "
#     f"    (args->>'assetOut')::integer as asset_out,"
#     f"    (args->>'assetFeeAmount')::numeric as asset_fee,"
#     f"    (args->>'protocolFeeAmount')::numeric as protocol_fee,"
#     f"    (args->>'amountOut')::numeric as amount,"
#     f"    (args->>'amountIn')::numeric as amount_in"
#     f"from event "
#     f"inner join block on block_id = block.id"
#     f"where name like 'Omnipool.%Executed'"
#     f"order by block_id asc) as trades) as normalized_trades"
#     f"where asset_out = 'DOT' "
#     f")"
#     f"select * from fees WHERE ""timestamp"" BETWEEN '2024-01-03T14:43:30.767Z' AND '2024-01-10T14:43:30.767Z'"
# )

In [3]:
# query = (
#     f"WITH pair_price AS (select "
#     f"    timestamp,"
#     f"    amount_in / amount_out AS price"
#     f"FROM normalized_trades"
#     f"WHERE asset_in = 'DAI' AND asset_out = 'HDX' "
#     f"    AND ""timestamp"" BETWEEN '2023-07-14T19:28:59.673Z' AND '2023-07-21T19:28:59.673Z' "
#     f"UNION ALL"
#     f"SELECT "
#     f"    timestamp,"
#     f"    amount_out / amount_in AS price"
#     f"FROM normalized_trades"
#     f"WHERE asset_in = 'HDX' AND asset_out = 'DAI' "
#     f" AND ""timestamp"" BETWEEN '2023-07-14T19:28:59.673Z' AND '2023-07-21T19:28:59.673Z' "
#     f"ORDER BY timestamp)"
#     f"SELECT"
#     f"   timestamp AS ""time"","
#     f"   price"
#     f"FROM pair_price"
#     f"ORDER BY 1"
#     f"LIMIT 1000"
# )

In [4]:
query = (
    f"WITH normalized_trades AS ("
    f"  SELECT" 
    f"    timestamp,"
    f"    block,"
    f"    who,"
    f"    operation,"
    f"    (SELECT symbol FROM token_metadata WHERE id = asset_in LIMIT 1) AS asset_in,"
    f"    (SELECT symbol FROM token_metadata WHERE id = asset_out LIMIT 1) AS asset_out,"
    f"    amount_in / POWER(10, (SELECT decimals FROM token_metadata WHERE id = asset_in LIMIT 1)) AS amount_in,"
    f"    amount_out / POWER(10, (SELECT decimals FROM token_metadata WHERE id = asset_out LIMIT 1)) AS amount_out"
    f"  FROM ("
    f"    SELECT" 
    f"      timestamp,"
    f"      block.height AS block,"
    f"      args->>'who' AS who,"
    f"      name AS operation,"
    f"      (args->>'assetIn')::integer AS asset_in," 
    f"      (args->>'assetOut')::integer AS asset_out,"
    f"      (args->>'amountIn')::numeric AS amount_in," 
    f"      (args->>'amountOut')::numeric AS amount_out"
    f"    FROM event" 
    f"    INNER JOIN block ON block_id = block.id"
    f"    WHERE name LIKE 'Omnipool.%Executed'"
    f"      AND ""timestamp"" BETWEEN '2024-01-01T00:00:00.000Z' AND '2024-01-29T00:00:00.000Z'" 
    f"  ) AS trades"
    f"),"
    f"trades AS ("
    f"  SELECT" 
    f"    timestamp,"
    f"    amount_in / amount_out AS price,"
    f"    amount_in as amount,"
    f"    'buy HDX' as direction"
    f"  FROM normalized_trades"
    f"  WHERE asset_in = 'USDT' AND asset_out = 'HDX'" 
    f"  UNION ALL"
    f"  SELECT" 
    f"    timestamp,"
    f"    amount_out / amount_in AS price,"
    f"    amount_out as amount,"
    f"    'sell HDX' as direction"
    f"  FROM normalized_trades"
    f"  WHERE asset_in = 'HDX' AND asset_out = 'USDT'"
    f")"
    f"SELECT"
    f"  timestamp AS ""time"","
    f"  price,"
    f"  amount,"
    f"  direction "
    f"FROM trades "
    f"ORDER BY 1; "
)

In [5]:
# query = (
#    f"select sum(amt) from"
#    f"("
#    f"    SELECT *, CAST(args->>'tradeRewards' as bigint) as amt from event"
#    f"    where event.name = 'Referrals.Claimed'"
#    f"    and event.args->>'who' = '0x6d6f646c7374616b696e67230000000000000000000000000000000000000000'  -- staking"
#    f") tempTable"
# )

In [6]:
# query = (
#     f"select data.extrinsic_id, data.call_id, data.call_name, data.event_name, data.call_success,"
#     f"    data.event_args->>'asset' as asset,"
#     f"    data.event_args->>'price' as price,"
#     f"    data.event_args->>'amount' as amount,"
#     f"    data.event_args->>'shares' as shares,"
#     f"    data.event_args->>'positionId' as position_id,"
#     f"    data.event_args->>'owner' as who,"
#     f"    fees.fee, fees.tip"
#     f"from"
#     f"(select call.extrinsic_id, call.name as call_name, call.args as call_args, call.success as call_success, event.call_id,"
#     f"event.name as event_name, event.args as event_args"
#     f"from event inner join call"
#     f"on call.id = event.call_id"
#     f"and call.name = 'Omnipool.add_liquidity'"
#     f") as data"
#     f"inner join"
#     f"(select fee, tip, id from extrinsic) as fees"
#     f"on data.extrinsic_id = fees.id"
#     f"where event_name like 'Omnipool.PositionCreated'"
#     f"and cast(SUBSTR(data.extrinsic_id, 4, 7) as integer) > 3000000"
#     f"order by extrinsic_id"
#     f"LIMIT 100"
# )

In [ ]:
data = await query_sqlPad(query)
print("done")

waiting for query to finish...


In [ ]:
plt.figure(figsize=(20, 5))
plt.plot([line[1] for line in data])
plt.title('HDX/USDT price over time')

In [ ]:
buy_sell_ratio = 0
ratio_trend = []
smooth_factor = 20
decay_factor = 1 / smooth_factor
for line in data:
    if line[3].startswith('buy'):
        buy_sell_ratio = buy_sell_ratio * (1 - decay_factor) + decay_factor
    else:
        buy_sell_ratio = buy_sell_ratio * (1 - decay_factor) - decay_factor
    ratio_trend.append(buy_sell_ratio)
plt.figure(figsize=(20, 5))
plt.plot(ratio_trend)
plt.title('HDX/USDT buy/sell ratio over the same time period')

In [ ]:
print(query)